In [8]:
!pip install spacy
!pip install nltk
!pip install requests
!python -m spacy download en_core_web_sm


     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     --------------------------------------- 0.0/12.8 MB 326.8 kB/s eta 0:00:40
     --------------------------------------- 0.1/12.8 MB 798.9 kB/s eta 0:00:16
     - -------------------------------------- 0.4/12.8 MB 1.9 MB/s eta 0:00:07
     -- ------------------------------------- 0.7/12.8 MB 2.8 MB/s eta 0:00:05
     ---- ----------------------------------- 1.4/12.8 MB 4.8 MB/s eta 0:00:03
     -------- ------------------------------- 2.7/12.8 MB 8.1 MB/s eta 0:00:02
     ------------ --------------------------- 3.9/12.8 MB 10.2 MB/s eta 0:00:01
     ----------------- ---------------------- 5.5/12.8 MB 13.0 MB/s eta 0:00:01
     ------------------- -------------------- 6.2/12.8 MB 13.3 MB/s eta 0:00:01
     ----------------------- ---------------- 7.5/12.8 MB 14.6 MB/s eta 0:00:01
     ------------------------- -------------- 8.3/12.8 MB 14

IMport Required libraries

In [10]:
import spacy
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk import ne_chunk, pos_tag
import requests

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('averaged_perceptron_tagger')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

Fetching a News Article

In [11]:
# Replace 'YOUR_API_KEY' with your actual API key from News API
api_key = 'b3b55462a534432fabf4957334cb6093'
url = f'https://newsapi.org/v2/top-headlines?country=us&apiKey={api_key}'

response = requests.get(url)
news_data = response.json()

# Get the content of the first article
article = news_data['articles'][0]['content']
print(article)


When a 15-year-old girl I see in psychotherapy ended up in the ER with second-degree burns, she was embarrassed to tell me why. Eventually she disclosed having crouched over boiling herb-infused wate… [+132 chars]


Extracting Entities with NLTK (Rule-Based Approach)

In [12]:
def nltk_ner(text):
    sentences = sent_tokenize(text)
    tokens = [word_tokenize(sent) for sent in sentences]
    pos_tagged = [pos_tag(token) for token in tokens]
    chunked = [ne_chunk(tag) for tag in pos_tagged]
    
    entities = []
    for tree in chunked:
        for subtree in tree:
            if isinstance(subtree, nltk.Tree):
                entity_name = " ".join([token for token, pos in subtree.leaves()])
                entity_type = subtree.label()
                entities.append((entity_name, entity_type))
    
    return entities

nltk_entities = nltk_ner(article)
print("Entities extracted using NLTK:", nltk_entities)


Entities extracted using NLTK: [('ER', 'ORGANIZATION')]


Extract Entities using SpaCy (ML-Based Approach)

In [13]:
nlp = spacy.load('en_core_web_sm')

def spacy_ner(text):
    doc = nlp(text)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    return entities

spacy_entities = spacy_ner(article)
print("Entities extracted using SpaCy:", spacy_entities)


Entities extracted using SpaCy: [('15-year-old', 'DATE'), ('ER', 'ORG'), ('second', 'ORDINAL')]


Compare Results from SpaCy and NLTK

In [14]:
def compare_entities(nltk_entities, spacy_entities):
    nltk_set = set(nltk_entities)
    spacy_set = set(spacy_entities)
    
    only_nltk = nltk_set - spacy_set
    only_spacy = spacy_set - nltk_set
    common = nltk_set & spacy_set
    
    return only_nltk, only_spacy, common

only_nltk, only_spacy, common = compare_entities(nltk_entities, spacy_entities)

print("Entities only in NLTK:", only_nltk)
print("Entities only in SpaCy:", only_spacy)
print("Common Entities:", common)


Entities only in NLTK: {('ER', 'ORGANIZATION')}
Entities only in SpaCy: {('second', 'ORDINAL'), ('15-year-old', 'DATE'), ('ER', 'ORG')}
Common Entities: set()
